In [10]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [11]:
import os,glob
g = os.getcwd()
g
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

# Read data base and merge

- Read the base line and create ID

In [12]:
Points = gpd.read_file("Data_IDB/new/Zona_influencia_DR.shp")
# print(Points.crs)
Points = Points.to_crs("EPSG:4326")
ID = str(Points["cuest_id_b"]) + str(Points["parcela"])
str1 = [str(int) for int in Points["cuest_id_b"]] 
str2 = [str(int) for int in Points["parcela"]]
Points["ID"] = [int(s1 + s2) for s1, s2 in zip(str1, str2)]
ok = Points[Points['corregido2'].str.contains("ok")]
base = ok[["ID",'temporary_','patca']]
base["time"] = "Baseline"
# Points.to_csv("test.csv")
# Points = eeconvert.gdfToFc(Points)
# ok.columns
len(base)
base[base.ID == 386511]

<ipython-input-12-6e25b4b666df>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base["time"] = "Baseline"


,ID,temporary_,patca,time
29,386511,0,1,Baseline


- Read the follow-Up-1, create ID and remove duplicate ID

In [13]:
followup = pd.read_csv("Data_IDB/Panel_data_PATCA-DR-only_FU1.csv", encoding='latin-1')

In [14]:
followup[followup["cuest_id_b"]==3863]
# print(follwup.columns)
str1 = [str(int) for int in followup["cuest_id_b"]] 
str2 = [str(int) for int in followup["parcela"]]
followup["ID"] = [int(s1 + s2) for s1, s2 in zip(str1, str2)]
FU1 = followup[["ID","time",'temporary_crop']]
print(len(FU1))
ID = FU1["ID"].drop_duplicates()
FU1 = FU1.loc[ID.index.tolist()]
print(len(FU1))
FU1[FU1.ID == 386511]


646
441


,ID,time,temporary_crop
473,386511,1. Follow-up1,1


- Merge dateset

In [15]:
merge = base.merge(FU1,how='inner', on="ID")
print(len(merge))
merge.head()

377


,ID,temporary_,patca,time_x,time_y,temporary_crop
0,386311,1,1,Baseline,1. Follow-up1,1
1,386512,1,1,Baseline,1. Follow-up1,0
2,396011,1,0,Baseline,1. Follow-up1,1
3,407811,0,1,Baseline,1. Follow-up1,0
4,411411,1,0,Baseline,1. Follow-up1,1


- Select ID with trend

In [16]:
merge["logical"] = merge.temporary_ ==  merge.temporary_crop
trends = merge[merge.logical==True]
no_trends = merge[merge.logical!=True]
print(len(trends))
trends.head()

277


,ID,temporary_,patca,time_x,time_y,temporary_crop,logical
0,386311,1,1,Baseline,1. Follow-up1,1,True
2,396011,1,0,Baseline,1. Follow-up1,1,True
3,407811,0,1,Baseline,1. Follow-up1,0,True
4,411411,1,0,Baseline,1. Follow-up1,1,True
7,372211,1,0,Baseline,1. Follow-up1,1,True


In [28]:
no_trends[no_trends.ID==396011]
# type(trends.ID[0])

,ID,temporary_,time_x,time_y,temporary_crop,logical


In [29]:
trends[trends.ID==396011]

,ID,temporary_,time_x,time_y,temporary_crop,logical
2,396011,1,Baseline,1. Follow-up1,1,True
